<a href="https://colab.research.google.com/github/josephxlp/GEOM180-/blob/main/BareGround_EO_Sentinel2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: clip to angola:
# import ee
# import geemap
# # Initialize Earth Engine
# try:
#     ee.Initialize(project='ee-josephluntadilap')
# except Exception as e:
#     ee.Authenticate()
#     ee.Initialize(project='ee-josephluntadilap')
# # Load the datasets
# bare_surface = ee.Image('users/geocis/BareSurfaces/BS_1980_2019')
# bare_frequency = ee.Image('users/geocis/BareSurfaces/BF_1980_2019')
# # Define visualization parameters
# bare_surface_vis = {
#     'min': 0,
#     'max': 39,
#     'palette': ['white', 'red']
# }
# bare_frequency_vis = {
#     'min': 0,
#     'max': 1,
#     'palette': ['blue', 'green']
# }
# # Create a map centered on Angola
# Map = geemap.Map(center=[-12.5, 18.5], zoom=5)

# Clip the datasets to Angola
angola = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(ee.Filter.eq('country_na', 'Angola'))

bare_surface_angola = bare_surface.clip(angola)
bare_frequency_angola = bare_frequency.clip(angola)

# Add the clipped layers to the map
Map.addLayer(bare_surface_angola, bare_surface_vis, 'Bare Surface (Angola)')
Map.addLayer(bare_frequency_angola, bare_frequency_vis, 'Bare Frequency (Angola)')

# Display the map
Map


In [5]:


import ee
import geemap

# Initialize Earth Engine
try:
    ee.Initialize(project='ee-josephluntadilap')
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='ee-josephluntadilap')

# Load the datasets
bare_surface = ee.Image('users/geocis/BareSurfaces/BS_1980_2019')
bare_frequency = ee.Image('users/geocis/BareSurfaces/BF_1980_2019')

# Define visualization parameters
bare_surface_vis = {
    'min': 0,
    'max': 39,
    'palette': ['white', 'red']
}

bare_frequency_vis = {
    'min': 0,
    'max': 1,
    'palette': ['blue', 'green']
}

# Clip the datasets to Angola
angola = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(ee.Filter.eq('country_na', 'Angola'))

Map = geemap.Map(center=[-12.5, 18.5], zoom=5)
bare_surface_angola = bare_surface.clip(angola)
bare_frequency_angola = bare_frequency.clip(angola)

# Add the clipped layers to the map
Map.addLayer(bare_surface_angola, bare_surface_vis, 'Bare Surface (Angola)')
Map.addLayer(bare_frequency_angola, bare_frequency_vis, 'Bare Frequency (Angola)')


EEException: Image.visualize: Cannot provide a palette when visualizing more than one band.

In [4]:
bare_frequency

In [7]:
import ee
import geemap


# Step 1: Data Acquisition & Harmonization
# Load Landsat collections
landsat4 = ee.ImageCollection("LANDSAT/LT04/C01/T1_SR")
landsat5 = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")
landsat7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")
landsat8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")

# Function to rename bands to common names
def rename_bands(img, sensor):
    band_names = {
        'LT04': ["B1", "B2", "B3", "B4", "B5", "B7"],
        'LT05': ["B1", "B2", "B3", "B4", "B5", "B7"],
        'LE07': ["B1", "B2", "B3", "B4", "B5", "B7"],
        'LC08': ["B2", "B3", "B4", "B5", "B6", "B7"]
    }
    common_names = ["Blue", "Green", "Red", "NIR", "SWIR1", "SWIR2"]
    return img.select(band_names[sensor], common_names)

# Harmonize collections
landsat4 = landsat4.map(lambda img: rename_bands(img, 'LT04'))
landsat5 = landsat5.map(lambda img: rename_bands(img, 'LT05'))
landsat7 = landsat7.map(lambda img: rename_bands(img, 'LE07'))
landsat8 = landsat8.map(lambda img: rename_bands(img, 'LC08'))

# Merge collections
landsat_all = landsat4.merge(landsat5).merge(landsat7).merge(landsat8)

# Step 2: Bare Surface Detection
# Compute indices
def compute_indices(img):
    ndvi = img.normalizedDifference(["NIR", "Red"]).rename("NDVI")
    nbr2 = img.normalizedDifference(["SWIR2", "SWIR1"]).rename("NBR2")
    vnsir = img.expression(
        "(Red + Green + Blue) / (SWIR1 + SWIR2)",
        {"Red": img.select("Red"), "Green": img.select("Green"), "Blue": img.select("Blue"), "SWIR1": img.select("SWIR1"), "SWIR2": img.select("SWIR2")}
    ).rename("VNSIR")
    return img.addBands([ndvi, nbr2, vnsir])

landsat_all = landsat_all.map(compute_indices)

# Apply thresholds to detect bare surfaces
def mask_bare_surfaces(img):
    bs_mask = (img.select("NDVI").gte(-0.25).And(img.select("NDVI").lte(0.25))).And(
        img.select("NBR2").gte(-0.30).And(img.select("NBR2").lte(0.10))
    ).And(img.select("VNSIR").lt(0.9))
    return img.updateMask(bs_mask)

bare_surfaces = landsat_all.map(mask_bare_surfaces)

# Step 3: Bare Surface Frequency Calculation
def compute_bs_frequency(img):
    valid_mask = img.mask()
    bs_pixels = img.mask().Not().rename("BSF")
    return bs_pixels.addBands(valid_mask)

bs_frequency = bare_surfaces.map(compute_bs_frequency)

# Step 4: Temporal Analysis using Mann-Kendall Test
# (Simplified version, full MK test requires external libraries)

def compute_mk_trend(image_collection):
    def mk_test(img):
        year = ee.Number(img.date().get('year'))
        return img.addBands(ee.Image(year).rename('Year'))

    return image_collection.map(mk_test)

temporal_trends = compute_mk_trend(bs_frequency)

# Step 5: Area Calculation per Country
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")

def calculate_area(image):
    area_stats = image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=countries.geometry(),
        scale=250,
        bestEffort=True
    )
    return ee.Feature(None, area_stats)

area_results = temporal_trends.map(calculate_area)

# Display results
Map = geemap.Map()
Map.addLayer(bare_surfaces.median(), {'bands': ['Red', 'Green', 'Blue'], 'min': 0, 'max': 3000}, "Bare Surfaces")
Map.centerObject(countries, 2)
Map


EEException: ImageCollection.load: ImageCollection asset 'LANDSAT/LT05/C01/T1_SR' not found (does not exist or caller does not have access).

In [9]:
import ee
import geemap

# # Initialize the GEE
# try:
#     ee.Initialize()
# except Exception as e:
#     ee.Authenticate()
#     ee.Initialize()

# Step 1: Data Acquisition & Harmonization
# Load Harmonized Sentinel-2 collection
sentinel2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")

# Function to rename bands to common names
def rename_bands(img):
    common_names = ["Blue", "Green", "Red", "NIR", "SWIR1", "SWIR2"]
    return img.select(["B2", "B3", "B4", "B8", "B11", "B12"], common_names)

# Harmonize collection
sentinel2 = sentinel2.map(rename_bands)

# Step 2: Bare Surface Detection
# Compute indices
def compute_indices(img):
    ndvi = img.normalizedDifference(["NIR", "Red"]).rename("NDVI")
    nbr2 = img.normalizedDifference(["SWIR2", "SWIR1"]).rename("NBR2")
    vnsir = img.expression(
        "(Red + Green + Blue) / (SWIR1 + SWIR2)",
        {"Red": img.select("Red"), "Green": img.select("Green"), "Blue": img.select("Blue"), "SWIR1": img.select("SWIR1"), "SWIR2": img.select("SWIR2")}
    ).rename("VNSIR")
    return img.addBands([ndvi, nbr2, vnsir])

sentinel2 = sentinel2.map(compute_indices)

# Apply thresholds to detect bare surfaces
def mask_bare_surfaces(img):
    bs_mask = (img.select("NDVI").gte(-0.25).And(img.select("NDVI").lte(0.25))).And(
        img.select("NBR2").gte(-0.30).And(img.select("NBR2").lte(0.10))
    ).And(img.select("VNSIR").lt(0.9))
    return img.updateMask(bs_mask)

bare_surfaces = sentinel2.map(mask_bare_surfaces)

# Step 3: Bare Surface Frequency Calculation
def compute_bs_frequency(img):
    valid_mask = img.mask()
    bs_pixels = img.mask().Not().rename("BSF")
    return bs_pixels.addBands(valid_mask)

bs_frequency = bare_surfaces.map(compute_bs_frequency)

# Step 4: Temporal Analysis using Mann-Kendall Test
# (Simplified version, full MK test requires external libraries)

def compute_mk_trend(image_collection):
    def mk_test(img):
        year = ee.Number(img.date().get('year'))
        return img.addBands(ee.Image(year).rename('Year'))

    return image_collection.map(mk_test)

# temporal_trends = compute_mk_trend(bs_frequency)

# # Step 5: Area Calculation per Country
# countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")

# def calculate_area(image):
#     area_stats = image.reduceRegion(
#         reducer=ee.Reducer.sum(),
#         geometry=countries.geometry(),
#         scale=250,
#         bestEffort=True
#     )
#     return ee.Feature(None, area_stats)

# area_results = temporal_trends.map(calculate_area)

# Display results
Map = geemap.Map()
Map.addLayer(bare_surfaces.median(), {'bands': ['Red', 'Green', 'Blue'], 'min': 0, 'max': 3000}, "Bare Surfaces")
Map.centerObject(countries, 2)
Map


EEException: Collection.geometry: Geometry has too many edges (3970390 > 2000000).

In [11]:
# https://www.nature.com/articles/s41598-020-61408-1#Sec8
#https://glad.umd.edu/dataset/global-2010-bare-ground-30-m

In [12]:
import ee
import geemap

# # Initialize the GEE
# try:
#     ee.Initialize()
# except Exception as e:
#     ee.Authenticate()
#     ee.Initialize()

# Step 1: Data Acquisition & Harmonization
# Load Harmonized Sentinel-2 collection
sentinel2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")

# Function to rename bands to common names
def rename_bands(img):
    common_names = ["Blue", "Green", "Red", "NIR", "SWIR1", "SWIR2"]
    return img.select(["B2", "B3", "B4", "B8", "B11", "B12"], common_names)

# Harmonize collection
sentinel2 = sentinel2.map(rename_bands)

# Limit analysis to Tripoli, Libya
tripoli = ee.Geometry.Point([13.1913, 32.8872]).buffer(50000)  # 50km buffer around Tripoli
sentinel2 = sentinel2.filterBounds(tripoli)

# Step 2: Bare Surface Detection
# Compute indices
def compute_indices(img):
    ndvi = img.normalizedDifference(["NIR", "Red"]).rename("NDVI")
    nbr2 = img.normalizedDifference(["SWIR2", "SWIR1"]).rename("NBR2")
    vnsir = img.expression(
        "(Red + Green + Blue) / (SWIR1 + SWIR2)",
        {"Red": img.select("Red"), "Green": img.select("Green"), "Blue": img.select("Blue"), "SWIR1": img.select("SWIR1"), "SWIR2": img.select("SWIR2")}
    ).rename("VNSIR")
    return img.addBands([ndvi, nbr2, vnsir])

sentinel2 = sentinel2.map(compute_indices)

# Apply thresholds to detect bare surfaces
def mask_bare_surfaces(img):
    bs_mask = (img.select("NDVI").gte(-0.25).And(img.select("NDVI").lte(0.25))).And(
        img.select("NBR2").gte(-0.30).And(img.select("NBR2").lte(0.10))
    ).And(img.select("VNSIR").lt(0.9))
    return img.updateMask(bs_mask)

bare_surfaces = sentinel2.map(mask_bare_surfaces)

# # Step 3: Bare Surface Frequency Calculation
# def compute_bs_frequency(img):
#     valid_mask = img.mask()
#     bs_pixels = img.mask().Not().rename("BSF")
#     return bs_pixels.addBands(valid_mask)

# bs_frequency = bare_surfaces.map(compute_bs_frequency)

# Step 4: Temporal Analysis using Mann-Kendall Test
# (Simplified version, full MK test requires external libraries)

# def compute_mk_trend(image_collection):
#     def mk_test(img):
#         year = ee.Number(img.date().get('year'))
#         return img.addBands(ee.Image(year).rename('Year'))

#     return image_collection.map(mk_test)

# temporal_trends = compute_mk_trend(bs_frequency)

# Step 5: Area Calculation in Tripoli

# def calculate_area(image):
#     area_stats = image.reduceRegion(
#         reducer=ee.Reducer.sum(),
#         geometry=tripoli,
#         scale=250,
#         bestEffort=True
#     )
#     return ee.Feature(None, area_stats)

# area_results = temporal_trends.map(calculate_area)

# Display results
Map = geemap.Map()
Map.addLayer(bare_surfaces.median(), {'bands': ['Red', 'Green', 'Blue'], 'min': 0, 'max': 3000}, "Bare Surfaces")
Map.centerObject(tripoli, 10)
Map


Map(center=[32.88725129235148, 13.191301693863457], controls=(WidgetControl(options=['position', 'transparent_…